In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import gdown
import zipfile
import os
import json
import re

In [ ]:
url = 'https://drive.google.com/uc?id=1yIDmIeifB2_rwl6U45F23AAgnpSmnkbc'

# unzip the file and store dataset in recipe_box/
output = 'recipe_box.zip'
gdown.download(url, output, quiet=False)
extract_to = 'recipe_box'
os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

Downloading...
From (original): https://drive.google.com/uc?id=1yIDmIeifB2_rwl6U45F23AAgnpSmnkbc
From (redirected): https://drive.google.com/uc?id=1yIDmIeifB2_rwl6U45F23AAgnpSmnkbc&confirm=t&uuid=97e0a1f3-40ef-4375-993a-edd306ffe5de
To: /content/recipe_box.zip
100%|██████████| 53.4M/53.4M [00:01<00:00, 27.1MB/s]


In [ ]:
def clean_text(text):
    # Remove the string "ADVERTISEMENT"
    cleaned_text = re.sub(r'ADVERTISEMENT|\s+', ' ', text).strip()
    return cleaned_text

def clean_dataset(dataset):
    for recipe in dataset:
        if 'ingredients' in recipe:
            recipe['ingredients'] = [clean_text(ingredient) for ingredient in recipe['ingredients'] if ingredient]
        if 'instructions' in recipe and recipe['instructions']:
            recipe['instructions'] = clean_text(recipe['instructions'])
    return dataset

def recipe_validate_required_fields(recipe):
    required_keys = ['title', 'ingredients', 'instructions']
    if not recipe:
        return False
    for required_key in required_keys:
        if not recipe.get(required_key):
            return False
        if isinstance(recipe[required_key], list) and not recipe[required_key]:
            return False
    return True

In [ ]:
def load_dataset():
  dataset_file_names = [
    'recipes_raw_nosource_ar.json',
    # 'recipes_raw_nosource_epi.json',
    # 'recipes_raw_nosource_fn.json',
  ]

  dataset = []
  for dataset_file_name in dataset_file_names:
    dataset_file_path = os.path.join('recipe_box', dataset_file_name)
    with open(dataset_file_path, 'r') as dataset_file:
      json_data_dict = json.load(dataset_file)
      json_data_list = list(json_data_dict.values())
      json_data_list = [recipe for recipe in json_data_list if recipe_validate_required_fields(recipe)]
      cleaned_data = clean_dataset(json_data_list)
      dataset.extend(cleaned_data)
      print_dataset_info(dataset_file_path, cleaned_data)

  return dataset

def print_dataset_info(file_path, json_data):
  print(file_path)
  print('samples size:', len(json_data))
  if json_data:
    print('title:', json_data[0]['title'])
    print('ingredients:', json_data[0]['ingredients'])
    print('instructions:', json_data[0]['instructions'])
    print()

dataset_raw = load_dataset()

recipe_box/recipes_raw_nosource_ar.json
samples size: 39522
title: Slow Cooker Chicken and Dumplings
ingredients: ['4 skinless, boneless chicken breast halves', '2 tablespoons butter', '2 (10.75 ounce) cans condensed cream of chicken soup', '1 onion, finely diced', '2 (10 ounce) packages refrigerated biscuit dough, torn into pieces', '']
instructions: Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover. Cover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
vocab_size = 5000
max_sequence_length = 500  # Maximum length of sequences

# Tokenizer configuration
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
all_texts = [recipe['ingredients'] + [recipe['instructions']] for recipe in dataset_raw]
all_texts = [' '.join(texts) for texts in all_texts]  # Combine ingredients and instructions for tokenization
tokenizer.fit_on_texts(all_texts)

# Function to prepare sequences
def prepare_sequences(texts, tokenizer, max_sequence_length):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')
    return padded_sequences

# Preparing data
ingredients_seqs = [' '.join(recipe['ingredients']) for recipe in dataset_raw]
instructions_seqs = [recipe['instructions'] for recipe in dataset_raw]

X = prepare_sequences(ingredients_seqs, tokenizer, max_sequence_length)
y = prepare_sequences(instructions_seqs, tokenizer, max_sequence_length)

# Splitting data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)  # Should be (num_samples, sequence_length)
print("y_train shape:", y_train.shape)  # Should also be (num_samples, sequence_length)


X_train shape: (31617, 500)
y_train shape: (31617, 500)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model configuration

embedding_dim = 256
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    LSTM(256, return_sequences=True),
    # LSTM(32, return_sequences=True),
    Dense(vocab_size, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 500, 256)          1280000   
                                                                 
 lstm_14 (LSTM)              (None, 500, 256)          525312    
                                                                 
 dense_7 (Dense)             (None, 500, 5000)         1285000   
                                                                 
Total params: 3090312 (11.79 MB)
Trainable params: 3090312 (11.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Training configuration
epochs = 10
batch_size = 64

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_test, y_test))

Epoch 1/10
495/495 [==============================] - 115s 228ms/step - loss: 1.6713 - accuracy: 0.7953 - val_loss: 1.3232 - val_accuracy: 0.8010
Epoch 2/10
495/495 [==============================] - 119s 241ms/step - loss: 1.2973 - accuracy: 0.8016 - val_loss: 1.2684 - val_accuracy: 0.8037
Epoch 3/10
495/495 [==============================] - 119s 241ms/step - loss: 1.2668 - accuracy: 0.8027 - val_loss: 1.2514 - val_accuracy: 0.8041
Epoch 4/10
495/495 [==============================] - 120s 241ms/step - loss: 1.2499 - accuracy: 0.8034 - val_loss: 1.2427 - val_accuracy: 0.8050
Epoch 5/10
495/495 [==============================] - 119s 241ms/step - loss: 1.2371 - accuracy: 0.8040 - val_loss: 1.2304 - val_accuracy: 0.8051
Epoch 6/10
495/495 [==============================] - 119s 241ms/step - loss: 1.2239 - accuracy: 0.8044 - val_loss: 1.2257 - val_accuracy: 0.8054
Epoch 7/10
495/495 [==============================] - 113s 229ms/step - loss: 1.2121 - accuracy: 0.8047 - val_loss: 1.2172 -

In [ ]:
sample_ingredients = "chicken egg butter"

sample_seq = tokenizer.texts_to_sequences([sample_ingredients])
sample_padded = pad_sequences(sample_seq, maxlen=max_sequence_length, padding='post')

prediction = model.predict(sample_padded)

predicted_instruction_idx = prediction.argmax(axis=-1)[0]
predicted_instruction = tokenizer.sequences_to_texts([predicted_instruction_idx])

print("Generated Cooking Instructions:")
print(predicted_instruction)

1/1 [==============================] - 0s 374ms/step
Generated Cooking Instructions:
['preheat oven to 350 degrees f degrees c a a a a a the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>